# 第8章 装饰器

## 8.1 闭包
装饰器是一种语法糖，是闭包的语法糖。让我来看一下闭包：


In [3]:
def get_avg():
    list_item = []
    
    def to_avg(item):
        nonlocal list_item
        list_item.append(item)
        avg = sum(list_item)/len(list_item)
        return avg
    return to_avg
        
cal_avg = get_avg()
cal_avg(5)
cal_avg(2)

3.5

更接近装饰器的闭包写法：

In [12]:
import functools

def my_deco(foo):
    list_item = []
    
    @functools.wraps(foo) # 避免覆盖掉原函数的元数据
    def add_result(*arg, **kwargs):
        nonlocal list_item
        item = foo(*arg, **kwargs)
        list_item.append(item)
        print(list_item)
        return item
    return add_result

@my_deco
def sum_item(x,y):
    return x+y

sum_item(3,4)
sum_item(7,8)

[7]
[7, 15]


15

说装饰器是一种语法糖，是因为以上代码其实是运行了以下代码：

In [26]:
index_foo = my_deco(sum_item)
index_foo(5,6)

[7, 15, 11]
[11]


11

函数实现的装饰器，有一个知名的问题。那就是如果装饰类里的方法，因为类方法默认第一个参数是`self`，会出现问题。此时可以使用第三方库的`@wrapt.decorator`，解决此问题。

## 8.2 类的__call__实现装饰器

优点：更容易实现带参数的装饰器
注意：记得原函数的元数据；注意装饰器的括号

In [25]:
import functools
import time

class timer:
    """装饰器：输出装饰函数的执行时间
    
    param print_args:是否打印函数参数 
    """
    def __init__(self, print_args=True):
        self.print_args = print_args
        
    def __call__(self, func):
        @functools.wraps(func) # 避免覆盖掉原函数的元数据
        def print_time(*args, **kwargs):
            starttime = time.perf_counter()
            result = func(*args, **kwargs)
            
            if self.print_args:
                print(f"{func.__name__}")
            print(f"花费时间{time.perf_counter() - starttime}")
            return result
        return print_time
        
@timer() # 注意括号一定要写
def cal_add(x,y):
    return x+y

cal_add(10,12)

cal_add
花费时间0.0008414580001954164


22

## 8.3 类的实例实现装饰器

和上面实现不同的是，被装饰的function，传递给类的self实例，`__call__`方法看起来更加清晰符合直觉。

In [37]:
from functools import update_wrapper

class NewTimer:
    """实例实现装饰器
    """
    
    def __init__(self,function,* , print_args=True):
        # 避免覆盖掉原函数的元数据
        update_wrapper(self, function)
        self.function = function
        self.print_args = print_args
        
    def __call__(self, *args, **kwargs):
        starttime = time.perf_counter()
        result = self.function(*args, **kwargs)

        if self.print_args:
            print(f"{self.function.__name__}")
        print(f"花费时间{time.perf_counter() - starttime}")
        return result
    
    def add_foo(self):
        print("增加了新的特性")
        
@NewTimer # 注意括号不需要了
def cal_add(x,y):
    return x+y

cal_add(2,3)


cal_add
花费时间0.0003366249993632664


5

## 8.5 什么时候用装饰器？

1. 运行时校验：对函数执行前介入，校验不通过时终止执行
2. 注入额外参数:  `unittest.mock` 的装饰器`@patch`

编写装饰器，要注意一定要写的浅，把实现细节写到函数或者类中。可以参考`Click`库。

